In [1]:
import swyft

from forward import *

In [2]:
# a TMNRE prior function takes a random variable between 0 and 1 and produces an input vector from a prior of your choice

# by convention, nobs parameters preceed obs parameters

# define prior chunks
global_low_list = [65.0]
global_high_list = [75.0]
low_list = [0.0]*n_events
high_list = [1.0]*n_events

# append prior chunks
low_list = global_low_list + low_list
high_list = global_high_list + high_list

# instantiate prior
low = np.array(low_list)
high = np.array(high_list)
prior = swyft.get_uniform_prior(low, high)

# save prior
prior.save(prior_filename)

In [3]:
observation_o = {'x': np.array([1.0])}

n_observation_features = observation_o[observation_key].shape[0]
observation_shapes = {key: value.shape for key, value in observation_o.items()}

simulator = swyft.Simulator(
    forward,
    n_parameters,
    sim_shapes=observation_shapes
)

In [4]:
# set up storage

store = swyft.Store.memory_store(simulator)
store.add(n_training_samples, prior)
store.simulate()

dataset = swyft.Dataset(n_training_samples, prior, store)

Creating new store.
Store: Adding 2 new samples to simulator store.
Input H0: 70.84921264648438

Basic model file:
  C:\Users\sgagn\Anaconda3\envs\kn\Lib\site-packages\mosfit\models\model.json
Model file:
  C:\Users\sgagn\Anaconda3\envs\kn\Lib\site-packages\mosfit\models\bns_generative\bns_generative.json
Parameter file:
  models\bns_generative\parameters.json
No event specified, generating dummy data.

Loading data...

Loading module `transient` [ 0% ]
Constructing observation array [ 0.0% ]
Constructing observation array [ 25.0% ]
Constructing observation array [ 50.0% ]
Constructing observation array [ 75.0% ]

Loading module `lumdist` [ 2% ]

Loading module `redshift` [ 3% ]

Loading module `alltimes` [ 5% ]

Loading module `texplosion` [ 7% ]

Loading module `resttimes` [ 8% ]

Loading module `Mchirp` [ 10% ]

Loading module `q` [ 12% ]

Loading module `disk_frac` [ 13% ]

Loading module `radius_ns` [ 15% ]

Loading module `alpha` [ 17% ]

Loading module `Mtov` [ 18% ]

Loading mo

In [8]:
list(dataset)

[({'x': tensor([0.8375])},
  tensor([0.5849, 0.7326]),
  tensor([70.8492,  0.7326])),
 ({'x': tensor([0.])}, tensor([0.1674, 0.2038]), tensor([66.6739,  0.2038]))]

In [9]:
dataset.save('a1/pingas.pt')

In [19]:
store2 = swyft.Store.memory_store(simulator)

dl = swyft.Dataset.load(
    filename='a1/pingas.pt',
    store=store2
)

print(len(dl))

store2.add(len(dl), prior)

Creating new store.
2
Store: Adding 2 new samples to simulator store.


In [16]:
# len(dl), list(dl)
len(store2)

0

In [ ]:
def do_round_1d(bound, observation_focus):
    store.add(n_training_samples, prior, bound=bound)
    store.simulate()

    dataset = swyft.Dataset(n_training_samples, prior, store, bound = bound)

    # save dataset
    dataset.save(dataset_filename)

    network_1d = swyft.get_marginal_classifier(
        observation_key=observation_key,
        marginal_indices=marginal_indices_1d,
        observation_shapes=observation_shapes,
        n_parameters=n_parameters,
        hidden_features=32,
        num_blocks=2,
    )
    mre_1d = swyft.MarginalRatioEstimator(
        marginal_indices=marginal_indices_1d,
        network=network_1d,
        device=device,
    )
    mre_1d.train(dataset)

    mre_1d.save(mre_1d_filename)

    posterior_1d = swyft.MarginalPosterior(mre_1d, prior, bound)
    new_bound = posterior_1d.truncate(n_posterior_samples_for_truncation, observation_focus)
    
    # save new bound
    new_bound.save(bound_filename)

    return posterior_1d, new_bound

bound = None
for i in range(1):
    tic = time.time()
    posterior_1d, bound = do_round_1d(bound, observation_o)
    toc = time.time()

    elapsed = toc-tic
    tlist += [elapsed]
    np.savetxt('time.txt', np.array(tlist))

    dataset = swyft.Dataset(n_training_samples, prior, store)


tic = time.time()

network_1d = swyft.get_marginal_classifier(
    observation_key=observation_key,
    marginal_indices=marginal_indices_1d,
    observation_shapes=observation_shapes,
    n_parameters=n_parameters,
    hidden_features=32,
    num_blocks=2,
)
mre_1d = swyft.MarginalRatioEstimator(
    marginal_indices=marginal_indices_1d,
    network=network_1d,
    device=device,
)
mre_1d.train(dataset)

store.add(n_training_samples + 100, prior, bound=bound)
store.simulate()
dataset = swyft.Dataset(n_training_samples, prior, store, bound = bound)

mre_1d.train(dataset)

toc = time.time()
elapsed = toc-tic
tlist += [elapsed]
np.savetxt('time.txt', np.array(tlist))

dataset = swyft.Dataset(n_training_samples, prior, store)

# SAVING

prior.save(prior_filename)
dataset.save(dataset_filename)
mre_1d.save(mre_1d_filename)
bound.save(bound_filename)


n_rejection_samples = 10000

print("producing posterior")

posterior_1d = swyft.MarginalPosterior(mre_1d, prior, bound=bound)

print("sampling")

samples_1d = posterior_1d.sample(n_rejection_samples, observation_o)
key = marginal_indices_1d[0]

print(samples_1d[key])

plt.hist(samples_1d[key], 100)
plt.show()

np.savetxt("H0_samples.txt", samples_1d[key])